In [ ]:
from cookbook.interface import PluginInstanceRedisInterface
import os
import sys

import logging

logging.basicConfig()

# Set up redis credentials
redis_host = 'redis'
redis_port = 6379
redis_password = ''
redis_channel = os.environ.get("REDIS_CHANNEL")

# Increase the recursion limit in order to properly serialize Complexes
recursion_limit = 10000
sys.setrecursionlimit(recursion_limit)

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
comp_list = plugin_instance.request_complex_list()


In [ ]:
# from rmsd_v2.RMSDV2 import RMSDV2
from rmsd_v2.fpocket_client import FPocketClient

# redis_host, redis_port, redis_password, redis_channel
# rmsd_v2 = RMSDV2Redis()
ws = plugin_instance.request_workspace()
comp = ws.complexes[0]

fpocket_client = FPocketClient()
pockets = fpocket_client.get_pockets(comp)
